Let's try some basic tests of `VB.jl`:

In [1]:
include("VB.jl")
using VB
using Distributions

# Make a node

In [2]:
μ = rand(5, 5)
σ = rand(5, 5)
aa = RandomNode(:a, [:i, :j], Normal, μ, σ)
bb = RandomNode(:b, [:j, :k], Gamma, rand(5, 3), rand(5, 3))

VB.RandomNode{Distributions.Gamma}(:b,[:j,:k],5x3 Array{Distributions.Gamma,2}:
 Distributions.Gamma(α=0.28797366136020197, θ=0.36229129671396176)   …  Distributions.Gamma(α=0.17405770026310874, θ=0.4715545515324626)
 Distributions.Gamma(α=0.033815470745805376, θ=0.02925160690571227)     Distributions.Gamma(α=0.2127655143692322, θ=0.21492302099117366)
 Distributions.Gamma(α=0.9986712283436712, θ=0.6541169080105216)        Distributions.Gamma(α=0.8910758944267789, θ=0.07474774716357402)
 Distributions.Gamma(α=0.7180366683732657, θ=0.40060000929797357)       Distributions.Gamma(α=0.9797349524042074, θ=0.14215853381704213)
 Distributions.Gamma(α=0.14307967355221618, θ=0.6233977022883894)       Distributions.Gamma(α=0.35719911187410114, θ=0.7532154780159808))

In [3]:
cc = ConstantNode(:c, [:i, :j], rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.67921    0.816944  0.0218626  0.728487
 0.0274611  0.517911  0.215145   0.444458
 0.715433   0.130255  0.373087   0.237667)

In [4]:
cc = ConstantNode(rand(3, 4), [:i, :j])

VB.ConstantNode{Float64}(symbol("##const#7733"),[:i,:j],3x4 Array{Float64,2}:
 0.428808  0.578368   0.514897  0.703365
 0.859959  0.0622464  0.988834  0.220225
 0.110366  0.772437   0.54893   0.80651 )

In [5]:
nodes = Node[aa, bb]
fi = get_structure(nodes...)
fi

VB.FactorInds([:i,:j,:k],[5,3,5],Dict(:a=>[1,2],:b=>[2,3]))

In [6]:
a[i, j] ~ Normal(μ, σ)

VB.RandomNode{Distributions.Normal}(:a,[:i,:j],5x5 Array{Distributions.Normal,2}:
 Distributions.Normal(μ=0.45028111357557177, σ=0.0658350902734941)   …  Distributions.Normal(μ=0.7593855677186776, σ=0.7030559924689803)
 Distributions.Normal(μ=0.3682617719227985, σ=0.5437498162157315)       Distributions.Normal(μ=0.8757450187565858, σ=0.7324270777492721)
 Distributions.Normal(μ=0.4158074685790558, σ=0.18472090989997958)      Distributions.Normal(μ=0.5214744172693209, σ=0.8364054569181432)
 Distributions.Normal(μ=0.5139572814375037, σ=0.7797368671195515)       Distributions.Normal(μ=0.7884693659229178, σ=0.6578873677540704)
 Distributions.Normal(μ=0.04652781980624421, σ=0.28507036255517115)     Distributions.Normal(μ=0.7922040052489325, σ=0.859484738400832) )

In [7]:
c[i, j] ~ Const(rand(3, 4))

VB.ConstantNode{Float64}(:c,[:i,:j],3x4 Array{Float64,2}:
 0.0796852  0.995124  0.323916  0.0763783
 0.0619586  0.418177  0.14606   0.168157 
 0.207559   0.185558  0.460469  0.963826 )

# Make a factor

In [8]:
dims = (10, 2)

μ[j] ~ Normal(rand(dims[2]), ones(dims[2]))
τ ~ Gamma(1, 1)
x[i, j] ~ Normal(rand(dims), ones(dims))

f = @factor LogNormalFactor x μ τ;

# Make a (generated) value function

In [9]:
value(f)

-34.2450283244113

In [10]:
g = @factor EntropyFactor x
value(g)

28.378770664093462

# Let's make a simple model

We want a simple model of inference for a normal distribution:

$$
\begin{aligned}
    y \sim \mathcal{N}(\mu, \tau) \\
    \mu \sim \mathcal{N}(\mu_0, \tau_0) \\
    \tau \sim \mathrm{Ga}(\alpha, \beta) \\
    q(\mu) = \mathcal{N}(m, t) \\
    q(\tau) = \mathrm{Ga}(a, b)
\end{aligned}
$$

This model is conjugate, and can be solved by straightforward updates of the natural parameters of the posterior.

## Make the nodes

We need to make nodes for all the random variables that will need to be updated (we can also create nodes for parameter arrays, but these will be converted automatically if we don't).

In [11]:
dims = (20, 6)

# note: it won't matter much how we initialize here
μ[j] ~ Normal(zeros(dims[2]), ones(dims[2]))
τ[j] ~ Gamma(1.1 * ones(dims[2]), ones(dims[2]))
μ0[j] ~ Const(zeros(dims[2]))
τ0[j] ~ Const(2 * ones(dims[2]))
a0[j] ~ Const(1.1 * ones(dims[2]))
b0[j] ~ Const(ones(dims[2]))

y[i, j] ~ Const(rand(dims));

Now make factors: We need a Normal factor for the observation model plus a prior and an entropy for each node.

In [12]:
obs = @factor LogNormalFactor y μ τ
μ_prior = @factor LogNormalFactor μ μ0 τ0
τ_prior = @factor LogGammaFactor τ a0 b0

VB.LogGammaFactor{1}(VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.FactorInds([:j],[6],Dict(:τ=>[1],:b0=>[1],:a0=>[1])),Dict(:τ=>:x,:b0=>:β,:a0=>:α))

In [13]:
value(obs), value(μ_prior), value(τ_prior)

(-172.5668211295474,-13.593072740907873,0.0449816833123928)

In [14]:
m = VBModel([μ, τ, μ0, τ0, a0, b0, y], [obs, μ_prior, τ_prior])

VB.VBModel(VB.Node[VB.RandomNode{Distributions.Normal}(:μ,[:j],[Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0),Distributions.Normal(μ=0.0, σ=1.0)]),VB.RandomNode{Distributions.Gamma}(:τ,[:j],[Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0),Distributions.Gamma(α=1.1, θ=1.0)]),VB.ConstantNode{Float64}(:μ0,[:j],[0.0,0.0,0.0,0.0,0.0,0.0]),VB.ConstantNode{Float64}(:τ0,[:j],[2.0,2.0,2.0,2.0,2.0,2.0]),VB.ConstantNode{Float64}(:a0,[:j],[1.1,1.1,1.1,1.1,1.1,1.1]),VB.ConstantNode{Float64}(:b0,[:j],[1.0,1.0,1.0,1.0,1.0,1.0]),VB.ConstantNode{Float64}(:y,[:i,:j],20x6 Array{Float64,2}:
 0.255162   0.949913   0.764261   0.954222   0.630461   0.767552 
 0.342531   0.470748   0.239497   0.641009   0.101936   0.111441 
 0.488642   0.0199235  0.60681    0.960

In [15]:
[n.name for n in m.nodes]

7-element Array{Any,1}:
 :μ 
 :τ 
 :μ0
 :τ0
 :a0
 :b0
 :y 

In [16]:
[typeof(f) for f in m.factors]

3-element Array{DataType,1}:
 VB.LogNormalFactor{2}
 VB.LogNormalFactor{1}
 VB.LogGammaFactor{1} 

In [17]:
m.graph

Dict{VB.Node,Array{Tuple{VB.Factor{N},Symbol},1}} with 7 entries:
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{1}…
  VB.RandomNode{Distribut… => Tuple{VB.Factor{N},Symbol}[(VB.LogNormalFactor{2}…
  VB.ConstantNode{Float64… => Tuple{VB.Factor{N},Symbol}[(VB.LogGammaFactor{1}(…

In [18]:
naturals(obs, μ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.279317603470545,-11.000000000000002)
 (10.00779704017968,-11.000000000000002) 
 (11.295378796907194,-11.000000000000002)
 (10.882123543799786,-11.000000000000002)
 (12.158313566087239,-11.000000000000002)
 (10.939615855041415,-11.000000000000002)

In [19]:
naturals(obs, τ)

6-element Array{Tuple{Float64,Float64},1}:
 (10.0,12.954060138250975)
 (10.0,12.873537918079544)
 (10.0,13.371448627470127)
 (10.0,13.456892645340707)
 (10.0,13.684500546689065)
 (10.0,13.404554278397937)

In [20]:
check_conjugate(τ, m)

true

In [21]:
check_conjugate(μ, m)

true

In [22]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)
 Distributions.Normal(μ=0.0, σ=1.0)

In [23]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)
 Distributions.Gamma(α=1.1, θ=1.0)

In [27]:
update!(m)

In [28]:
μ.data

6-element Array{Distributions.Normal,1}:
 Distributions.Normal(μ=-0.5095994096677511, σ=0.7071067811865476)  
 Distributions.Normal(μ=-0.701593441202313, σ=0.7071067811865476)   
 Distributions.Normal(μ=0.2088643503117996, σ=0.7071067811865476)   
 Distributions.Normal(μ=-0.08335124152141189, σ=0.7071067811865476) 
 Distributions.Normal(μ=0.8190513773206577, σ=0.7071067811865476)   
 Distributions.Normal(μ=-0.042698038376367965, σ=0.7071067811865476)

In [29]:
τ.data

6-element Array{Distributions.Gamma,1}:
 Distributions.Gamma(α=26.313097705693558, θ=0.8999999999999999)
 Distributions.Gamma(α=30.178966725709653, θ=0.8999999999999999)
 Distributions.Gamma(α=17.662962746743922, θ=0.8999999999999999)
 Distributions.Gamma(α=20.350947401576953, θ=0.8999999999999999)
 Distributions.Gamma(α=17.33996715887029, θ=0.8999999999999999) 
 Distributions.Gamma(α=19.847421991938276, θ=0.8999999999999999)